# DBO (Dynamic Budget Optimization) implementation

### Import Statements

In [1]:
import time
from typing import List, Tuple, Dict

### Helper Functions

In [2]:
def compute_basic_metrics(s, m, b, r):
    """
    Compute core metrics for a single budget vector.

    s, m, b : lists of numbers (same length)
    r       : savings rate (0.0 to 1.0)
    """
    n = len(s)
    total_spend = sum(s)
    total_budget = sum(b)
    target_budget = (1.0 - r) * total_spend
    required_cut = total_spend - target_budget
    actual_cut = total_spend - total_budget
    target_error = total_budget - target_budget

    min_ok = all(b[i] + 1e-9 >= m[i] for i in range(n))
    target_reached = actual_cut + 1e-9 >= required_cut  # allow tiny float slack

    return {
        "total_spend": total_spend,
        "total_budget": total_budget,
        "target_budget": target_budget,
        "required_cut": required_cut,
        "actual_cut": actual_cut,
        "target_error": target_error,
        "min_ok": min_ok,
        "target_reached": target_reached,
    }

In [3]:
def print_budget_table(categories, s, m, p, b):
    """
    Nicely print a table of old vs new budgets for human reading.
    """
    n = len(s)
    if categories is None or len(categories) != n:
        categories = [f"C{i+1}" for i in range(n)]

    header = (
        f"{'Category':<15} {'Spend s[i]':>10} {'Min m[i]':>10} "
        f"{'Priority':>10} {'Budget b[i]':>12} {'Cut (s-b)':>12}"
    )
    print(header)
    print("-" * len(header))

    for i in range(n):
        cut = s[i] - b[i]
        print(
            f"{categories[i]:<15} "
            f"{s[i]:>10.2f} {m[i]:>10.2f} {p[i]:>10d} "
            f"{b[i]:>12.2f} {cut:>12.2f}"
        )

### Algorithm Implementation

In [4]:
def dbo_budget(
    s: List[float],
    m: List[float],
    p: List[int],
    r: float,
    unit: float = 10.0,
) -> Tuple[List[float], Dict]:
    """
    Dynamic Budget Optimization (DBO) using Dynamic Programming.

    Inputs
    ------
    s : list of last-month spends per category
    m : list of minimum acceptable budgets per category
    p : list of priorities (1 = highest protection, larger numbers = lower priority)
    r : global savings rate, e.g., 0.15 for 15%
    unit : discretization unit (same units as s, e.g., 10 currency units)

    Returns
    -------
    b : list of new budgets per category
    diagnostics : dict with metrics and internal stats
    """
    start_time = time.time()
    n = len(s)
    assert len(m) == n and len(p) == n, "s, m, p must have the same length"

    if unit <= 0:
        unit = 1.0

    # --- Step 1: totals and required cut ---
    total_spend = sum(s)
    target_budget = (1.0 - r) * total_spend
    required_cut_real = total_spend - target_budget

    ops_count = 0

    # --- Step 2: trivial case (no effective cut needed) ---
    if required_cut_real <= 0:
        b = [max(s[i], m[i]) for i in range(n)]
        ops_count += n

        diagnostics = compute_basic_metrics(s, m, b, r)
        diagnostics.update({
            "algorithm": "DBO",
            "required_cut_units": 0,
            "max_total_units": 0,
            "feasible_full_target": True,
            "dp_states": 0,
            "ops_count": ops_count,
            "runtime_ms": (time.time() - start_time) * 1000.0,
        })
        return b, diagnostics

    # --- Step 3: discretize required cut into units ---
    required_cut_units = int(round(required_cut_real / unit))
    if required_cut_units <= 0:
        # Rounding made it 0 units; treat as no-cut scenario
        b = [max(s[i], m[i]) for i in range(n)]
        ops_count += n

        diagnostics = compute_basic_metrics(s, m, b, r)
        diagnostics.update({
            "algorithm": "DBO",
            "required_cut_units": required_cut_units,
            "max_total_units": 0,
            "feasible_full_target": True,
            "dp_states": 0,
            "ops_count": ops_count,
            "runtime_ms": (time.time() - start_time) * 1000.0,
        })
        return b, diagnostics

    # --- Step 4: compute max possible cut units per category ---
    max_cut_units = [0] * n
    max_total_units = 0
    for i in range(n):
        discretionary = max(s[i] - m[i], 0.0)
        units_i = int(discretionary // unit)
        max_cut_units[i] = units_i
        max_total_units += units_i
        ops_count += 3

    # --- Step 5: feasibility check ---
    if max_total_units < required_cut_units:
        # Cannot reach target; cut everything to minimum
        b = [float(m[i]) for i in range(n)]
        ops_count += n

        diagnostics = compute_basic_metrics(s, m, b, r)
        diagnostics.update({
            "algorithm": "DBO",
            "required_cut_units": required_cut_units,
            "max_total_units": max_total_units,
            "feasible_full_target": False,
            "dp_states": 0,
            "ops_count": ops_count,
            "runtime_ms": (time.time() - start_time) * 1000.0,
        })
        return b, diagnostics

    # --- Step 6: penalty per unit (based on priority) ---
    penalty_per_unit = [float(pi) for pi in p]
    ops_count += n

    # --- Step 7: DP table definition ---
    # DP[i][c] = minimal penalty using first i categories to cut exactly c units
    U = required_cut_units
    INF = 10**18

    dp = [[INF] * (U + 1) for _ in range(n + 1)]
    choice = [[0] * (U + 1) for _ in range(n + 1)]
    dp[0][0] = 0
    dp_states = (n + 1) * (U + 1)

    # --- Step 8: fill DP table ---
    for i in range(1, n + 1):
        max_u_i = max_cut_units[i - 1]
        pen_i = penalty_per_unit[i - 1]
        for c in range(0, U + 1):
            best_pen = INF
            best_u = 0
            # try all units u that we can cut from category i-1
            max_u_here = min(max_u_i, c)
            for u in range(0, max_u_here + 1):
                prev = dp[i - 1][c - u]
                if prev == INF:
                    continue
                pen_here = prev + u * pen_i
                ops_count += 2
                if pen_here < best_pen:
                    best_pen = pen_here
                    best_u = u
            dp[i][c] = best_pen
            choice[i][c] = best_u

    # --- Step 9: reconstruct optimal solution ---
    cut_units = [0] * n
    c = U
    for i in range(n, 0, -1):
        u = choice[i][c]
        cut_units[i - 1] = u
        c -= u
        ops_count += 2

    # --- Step 10: compute final budgets ---
    b = [0.0] * n
    for i in range(n):
        cut_amount = cut_units[i] * unit
        bi = max(s[i] - cut_amount, m[i])
        b[i] = bi
        ops_count += 3

    # --- Step 11: finalize diagnostics ---
    basic = compute_basic_metrics(s, m, b, r)
    diagnostics = {
        "algorithm": "DBO",
        "required_cut_units": required_cut_units,
        "max_total_units": max_total_units,
        "feasible_full_target": True,
        "dp_states": dp_states,
        "ops_count": ops_count,
        "runtime_ms": (time.time() - start_time) * 1000.0,
    }
    diagnostics.update(basic)

    return b, diagnostics

### Example Demonstration

In [5]:
# Example scenario: simple monthly budget

categories = ["Rent", "Groceries", "Restaurants", "Transport", "Entertainment"]
s = [1200.0, 400.0, 350.0, 150.0, 200.0]   # last month spending
m = [1200.0, 300.0, 100.0, 100.0, 50.0]    # minimum acceptable budgets
p = [1, 2, 4, 3, 5]                        # 1 = highest protection
r = 0.15                                   # want to save 15%
unit = 10.0                                # DP unit size (10 currency units)

b_dbo, diag_dbo = dbo_budget(s, m, p, r, unit=unit)

print("=== DBO Budget Result ===")
print_budget_table(categories, s, m, p, b_dbo)
print("\nDiagnostics:")
for k, v in diag_dbo.items():
    print(f"{k:>20}: {v}")

=== DBO Budget Result ===
Category        Spend s[i]   Min m[i]   Priority  Budget b[i]    Cut (s-b)
--------------------------------------------------------------------------
Rent               1200.00    1200.00          1      1200.00         0.00
Groceries           400.00     300.00          2       300.00       100.00
Restaurants         350.00     100.00          4       160.00       190.00
Transport           150.00     100.00          3       100.00        50.00
Entertainment       200.00      50.00          5       200.00         0.00

Diagnostics:
           algorithm: DBO
  required_cut_units: 34
     max_total_units: 55
feasible_full_target: True
           dp_states: 210
           ops_count: 1909
          runtime_ms: 0.0
         total_spend: 2300.0
        total_budget: 1960.0
       target_budget: 1955.0
        required_cut: 345.0
          actual_cut: 340.0
        target_error: 5.0
              min_ok: True
      target_reached: False
